In [2]:
import pandas as pd
import numpy as np
import re

from textblob import TextBlob, Word

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk

In [3]:
df = pd.read_csv('raw_full_dataset.csv')

In [8]:
df.drop_duplicates(subset=['story_url'], inplace=True)
df.drop_duplicates(subset=['title'], inplace=True)
df = df[df.story_url != '-']
df = df[df.author_followers != '-']

df.reset_index(inplace=True, drop=True)

In [5]:
df.head()

,date,title,subtitle,claps,responses,author_handle,story_url,reading_time,text,author_followers,publication
0,01/01/2020,Making Python Programs Blazingly Fast,Let’s look at the performance of our Python pr...,3.4K,3,https://towardsdatascience.com/@martin.heinz,https://towardsdatascience.com/making-python-p...,5,Making Python Programs Blazingly Fast\n\nLet’s...,3.8K,NaN
1,01/01/2020,Implementing a fully convolutional network (FC...,"A tutorial on building, training and…",543,4,https://towardsdatascience.com/@himanshurawlani,https://towardsdatascience.com/implementing-a-...,11,Understanding and implementing a fully convolu...,331,NaN
2,01/01/2020,6 New Features in Python 3.8 for Python Newbies,Python Beginner,1.8K,4,https://towardsdatascience.com/@edenau,https://towardsdatascience.com/6-new-features-...,4,Python Beginner 6 New Features in Python 3.8 f...,1.7K,NaN
3,01/01/2020,How to be fancy with Python,Python tricks that will make your life easier,1.7K,12,https://towardsdatascience.com/@dipam44,https://towardsdatascience.com/how-to-be-fancy...,5,How to be fancy with Python\n\nPython is cool....,625,NaN
4,01/01/2020,Perfectly Privacy-Preserving AI,Data Privacy,0,0,https://towardsdatascience.com/@patriciathaine,-,10,-,-,NaN


In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [9]:
def clean_text(df, column_name):
    # lower case
    df[column_name] = df[column_name].apply(lambda x: " ".join(word.lower() for word in str(x).split()))
    
    # remove punctuation
    df[column_name] = df[column_name].str.replace('[^a-z ]', '')
    
    # this replaces two or more white spcaes with just one
    df[column_name] = df[column_name].str.replace(r"\s\s+",' ') 
    
    # remove stop words
    df[column_name] = df[column_name].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))
    
    # lemmatize
    df[column_name] = df[column_name].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))
    
    return df

In [10]:
df = clean_text(df, 'text')
df.head()

/var/folders/0g/2n4dpfnj6xg2x7twpn7nk6nh0000gn/T/ipykernel_66657/3869721140.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column_name] = df[column_name].str.replace('[^a-z ]', '')
/var/folders/0g/2n4dpfnj6xg2x7twpn7nk6nh0000gn/T/ipykernel_66657/3869721140.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column_name] = df[column_name].str.replace(r"\s\s+",' ')


,date,title,subtitle,claps,responses,author_handle,story_url,reading_time,text,author_followers,publication
0,01/01/2020,Making Python Programs Blazingly Fast,Let’s look at the performance of our Python pr...,3.4K,3,https://towardsdatascience.com/@martin.heinz,https://towardsdatascience.com/making-python-p...,5,making python program blazingly fast let look ...,3.8K,NaN
1,01/01/2020,Implementing a fully convolutional network (FC...,"A tutorial on building, training and…",543,4,https://towardsdatascience.com/@himanshurawlani,https://towardsdatascience.com/implementing-a-...,11,understanding implementing fully convolutional...,331,NaN
2,01/01/2020,6 New Features in Python 3.8 for Python Newbies,Python Beginner,1.8K,4,https://towardsdatascience.com/@edenau,https://towardsdatascience.com/6-new-features-...,4,python beginner new feature python python newb...,1.7K,NaN
3,01/01/2020,How to be fancy with Python,Python tricks that will make your life easier,1.7K,12,https://towardsdatascience.com/@dipam44,https://towardsdatascience.com/how-to-be-fancy...,5,fancy python python cool really cool however l...,625,NaN
4,01/01/2020,Perfectly Privacy-Preserving AI,Data Privacy,0,0,https://towardsdatascience.com/@patriciathaine,-,10,,-,NaN


In [23]:
def preprocess(df):
    stopwords = nltk.corpus.stopwords.words('english')
    df['title_process'] = df['title'].astype(str)
    df['title_process'] = df['title_process'].apply(lambda x : x.lower())
    df['title_process'] = df['title_process'].apply(lambda x : nltk.word_tokenize(x))
    df['title_process'] = df['title_process'].apply(lambda x : [item for item in x if item not in stopwords])
    df['title_process'] = df['title_process'].apply(lambda x : " ".join(x))
    df['title_process'] = df['title_process'].str.replace('@[^\s]+', "")
    df['title_process'] = df['title_process'].str.replace('https?:\/\/.*[\r\n]*', '')
    df['title_process'] = df['title_process'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df['title_process'] = df['title_process'].str.replace('\d+', '')
    df['title_process'] = df['title_process'].str.replace('[^\w\s]', '')
    return df

In [24]:
df = preprocess(df)

/var/folders/0g/2n4dpfnj6xg2x7twpn7nk6nh0000gn/T/ipykernel_9052/241149828.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title_process'] = df['title_process'].str.replace('@[^\s]+', "")
/var/folders/0g/2n4dpfnj6xg2x7twpn7nk6nh0000gn/T/ipykernel_9052/241149828.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title_process'] = df['title_process'].str.replace('https?:\/\/.*[\r\n]*', '')
/var/folders/0g/2n4dpfnj6xg2x7twpn7nk6nh0000gn/T/ipykernel_9052/241149828.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title_process'] = df['title_process'].str.replace('\d+', '')
/var/folders/0g/2n4dpfnj6xg2x7twpn7nk6nh0000gn/T/ipykernel_9052/241149828.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title_process'] = df['title_process'].str.replace('[^\w\s]', '')


In [27]:
print(df.text.iloc[76359])

the most beautiful theory of all

a century ago albert einstein changed the way humans saw the universe. his work is still offering new insights today

“alfred, it’s spinning.” roy kerr, a new zealand-born physicist in his late 20s, had, for half an hour, been chain-smoking his way through some fiendish mathematics. alfred schild, his boss at the newly built centre for relativity at the university of texas, had sat and watched. now, having broken the silence, kerr put down his pencil. he had been searching for a new solution to albert einstein’s equations of general relativity, and at last he could see in his numbers and symbols a precise description of how space-time — the four-dimensional universal fabric those equations describe — could be wrapped into a spinning ball. he had found what he was looking for.

when this happened, in 1962, the general theory of relativity had been around for almost half a century. it was customarily held up as one of the highest intellectual achievement

In [9]:
# lower case
df['text'] = df['text'].apply(lambda x: " ".join(word.lower() for word in str(x).split()))
# remove punctuation
df['text']

0        making python programs blazingly fast let’s lo...
1        understanding and implementing a fully convolu...
2        python beginner 6 new features in python 3.8 f...
3        how to be fancy with python python is cool. re...
4        from scratch to search: playing with your data...
                               ...                        
76359    the most beautiful theory of all a century ago...
76360    why people get happier as they get older as pe...
76361    is the economist left- or right-wing? neither....
76362    how the internet unleashed a burst of cartooni...
76363    remove paywalls from search results “for the c...
Name: text, Length: 76364, dtype: object